# Download Image Data from Bucket

In [ ]:
import sys
sys.path.append('../src')

from data_utils import download_images, display_tiff

# Bucket and directory
bucket_name = 'tree-counting-project'
prefix = 'switzerland/'
local_dir = '../Data/Switzerland'

# Download images
download_images(bucket_name, prefix, local_dir)

# Diplay Image

In [ ]:
# Display an image
tiff_path = '../Data/Switzerland/Lens_image.tif'
display_tiff(tiff_path)


In [1]:
import ee
ee.Authenticate()

True

In [2]:
ee.Initialize()

In [8]:
import geemap

In [10]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# https://viewer.nationalmap.gov/services/
url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
Map.add_tile_layer(url, name="Google Satellite", attribution="Google")